## START

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Loading Libraries

In [ ]:
! pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 5.1 MB/s 
     |████████████████████████████████| 451 kB 16.9 MB/s 
     |████████████████████████████████| 1.9 MB 47.8 MB/s 
     |████████████████████████████████| 43 kB 977 kB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 9.2 MB 44.9 MB/s 
     |████████████████████████████████| 5.8 MB 35.8 MB/s 
     |████████████████████████████████| 7.6 MB 41.5 MB/s 
     |████████████████████████████████| 182 kB 59.4 MB/s 
     |████████████████████████████████| 182 kB 49.0 MB/s 
     |████████████████████████████████| 168 kB 55.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 64.8 MB/s 
     |████████████████████████████████| 166 kB 59.3 MB/s 
     |████████████████████████████████| 166 kB 57.8 MB/s 
     |██████████████████████████████

In [ ]:
!pip install indic-nlp-library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 3.1 MB/s 
     |████████████████████████████████| 2.8 MB 10.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from indicnlp.tokenize import indic_tokenize  
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import itertools
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,SpatialDropout1D
from tensorflow.keras.layers import Embedding
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from simpletransformers.classification import ClassificationModel
from transformers import AutoTokenizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model


#import mxnet as mx
# from bert_embedding import BertEmbedding
# from sentence_transformers import SentenceTransformer

### Paths for data

In [ ]:
train_data_path=r"/content/drive/MyDrive/NLP_Project/constraint_Hindi_Train - Sheet1.csv"
test_data_path=r"/content/drive/MyDrive/NLP_Project/Test Set Complete - test.csv"
validation_data_path=r"/content/drive/MyDrive/NLP_Project/Constraint_Hindi_Valid - Sheet1.csv"

#### We will be following OOPS model

In [ ]:

class Hostility:
    #Loading the train, test and validation data in the constructor
    def __init__(self,train_data=train_data_path,test_data=test_data_path,validation_data=validation_data_path):
        self.df_data_train=pd.read_csv(train_data)
        self.df_data_test=pd.read_csv(test_data)
        self.df_data_validation=pd.read_csv(validation_data)
        self.stop_words=[]
        #Concatenating the datasets
        #self.df_combined=pd.concat([self.df_data_train,self.df_data_test,self.df_data_validation], ignore_index=True)
        self.df_combined=self.df_data_train.copy()
        self.labels=['hate', 'offensive', 'non-hostile', 'defamation', 'fake']
  
    
    
    def stopword_generation(self):
        stopword=open('/content/drive/MyDrive/NLP_Project/stopWords.txt','r')
        
        # pre-process stopword
        for i in stopword:
            i = re.sub('[\n]', '', i)
            self.stop_words.append(i)
        print(self.stop_words)
            
    def remove_mentions_and_URLs(self,dataframe):
        #self.df_combined['Post'] = self.df_combined["Post"].str.replace('http\S+|www.\S+|@\S+', '', case=False)
        #self.df_combined['Post'] = re.sub(r"(?:\@|https?\://)\S+", '', case=False)
        
        dataframe['Post'] = dataframe['Post'].apply(lambda i: re.sub(r"(?:\@|https?\://)\S+", '', i))
        return dataframe["Post"].head(5)
        
        
    def remove_hashtags(self,dataframe):
        dataframe['Post'] = dataframe["Post"].apply(lambda i:re.sub("#[A-Za-z0-9_]+","", i))
        return dataframe["Post"].head(5)
        

    #taken for stack overflow
    def remove_emojis(self,dataframe):
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  
            u"\U00002500-\U00002BEF"  
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  
            u"\u3030"
                          "]+", re.UNICODE)
        return re.sub(emoj, '', dataframe)
    
    def remove_newlines(self,dataframe):
        dataframe['Post']=dataframe["Post"].str.replace("\n","")
        return dataframe["Post"].tail(5)
    
    def remove_uneccesary_punctuation(self,dataframe):
        #re.sub('[^\u0900-\u097F]',' ',str(post))
        dataframe["Post"]=dataframe["Post"].apply(lambda i:re.sub('[^\u0900-\u097F]',' ',str(post)))
        return dataframe["Post"].tail(5)
    
    def tokenizer_using_indic(self):
        self.df_combined['tokenized_post'] = self.df_combined.apply(lambda row:indic_tokenize.trivial_tokenize(row['Post'], lang='hi'), axis=1)
        self.df_combined.drop(columns=['Post','Unique ID'])
        return self.df_combined.head(5)
    
    def remove_stopwords(self,dataframe):
        new_lines=""
        counter=0
        self.stopword_generation()
        for i in dataframe["Post"]:
            individual_words=i.split(' ')
            for j in individual_words:
                if j not in self.stop_words:
                    new_lines+=j
                    new_lines+=' '
            dataframe["Post"][counter]=new_lines
        counter+=1
        return dataframe["Post"].head(5)
        
    def Label_encoder_for_train(self):
        labels = self.df_combined['Labels Set'].apply(lambda x: list(x.split(",")))
        self.df_combined['Labels'] = labels
        multi_label=MultiLabelBinarizer()
        multi_label.fit(self.df_combined["Labels"])
        new_column_names = ["labels_%s" % c for c in multi_label.classes_]
        new_df=pd.DataFrame(multi_label.fit_transform(self.df_combined['Labels']),columns=multi_label.classes_)
        self.df_combined=pd.concat( [self.df_combined, new_df], axis=1 )
        #new_df.to_csv("train_test.csv")
        return self.df_combined.head(5)

    def Label_encoder_for_validation(self):
        labels = self.df_data_validation['Labels Set'].apply(lambda x: list(x.split(",")))
        self.df_data_validation['Labels'] = labels
        multi_label=MultiLabelBinarizer()
        multi_label.fit(self.df_data_validation["Labels"])
        new_column_names = ["labels_%s" % c for c in multi_label.classes_]
        new_df=pd.DataFrame(multi_label.fit_transform(self.df_data_validation['Labels']),columns=multi_label.classes_)
        self.df_data_validation=pd.concat( [self.df_data_validation, new_df], axis=1 )
        #new_df.to_csv("test_valid.csv")
        return self.df_data_validation.head(5)


    def Label_encoder_for_test(self):
        labels = self.df_data_test['Labels Set'].apply(lambda x: list(x.split(",")))
        self.df_data_test['Labels'] = labels
        multi_label=MultiLabelBinarizer()
        multi_label.fit(self.df_data_test["Labels"])
        new_column_names = ["labels_%s" % c for c in multi_label.classes_]
        new_df=pd.DataFrame(multi_label.fit_transform(self.df_data_test['Labels']),columns=multi_label.classes_)
        self.df_data_test=pd.concat( [self.df_data_test, new_df], axis=1 )
        return self.df_data_test.head(5)

    
        
        
    def tf_idf_vectorizer(self):
        y = self.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
        X = self.df_combined['Post']
        tfidfconverter = TfidfVectorizer(max_features=2000, min_df=2, max_df=0.9)  
        X = tfidfconverter.fit_transform(X).toarray()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.33, stratify=y)

    def Decision_Tree_classifier(self):
 
      label = ['hate', 'offensive', 'non-hostile', 'defamation', 'fake']
      for l in label:
          self.y_t1 = self.y_train[l]
          self.y_t2 = self.y_test[l]
          decision_tree = DecisionTreeClassifier(random_state=0, max_depth=15, class_weight='balanced')
          decision_tree = decision_tree.fit(self.X_train, self.y_t1)
          predictions = decision_tree.predict(self.X_test)
        # print(confusion_matrix(y_t2,predictions)) 
          print("For ",l," Tweets: Classification report: \n")
          print(classification_report(self.y_t2,predictions))


    def Decision_Tree_Classifier_for_tf_idf(self,weight,depth,criteria):
      import pickle
      for label in self.labels:
          clf = DecisionTreeClassifier(class_weight=weight, max_depth = depth, criterion=criteria).fit(self.X_train, self.y_train[label])
          pred = clf.predict(self.X_test)
          print(f"The model accuracy for class {label}:")
          print(classification_report(pred, self.y_test[label]))
          pickle.dump(clf,open('/content/drive/MyDrive/NLP_Project/'+(str(label)+"model.pkl"),'wb'))

    


    def Decision_Tree_Classifier_for_count_vectorizer(self,weight,depth,criteria):
      for label in self.labels:
          model_classifier= DecisionTreeClassifier(class_weight=weight, max_depth = depth, criterion=criteria).fit(self.Count_Vectorizer_X_train, self.Count_Vectorizer_Y_train[label])
          predictor = model_classifier.predict(self.Count_Vectorizer_X_test)
          print(f"The model accuracy for class {label}:")
          print(classification_report(predictor, self.Count_Vectorizer_Y_test[label]))
  

    def Count_Vectorizer(self):
        X = self.df_combined['Post']
        Y = self.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
        Count_Vectorizer_Y = self.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
        # Create our vectorizer
        #vectorizer = CountVectorizer(max_features=3000,min_df=2.0, max_df=0.8)
        vectorizer = CountVectorizer(max_features=3000,min_df=2, max_df=0.8)

        # Create the vectorizer
        Vectorized_X = vectorizer.fit_transform(X).toarray()
        df_X = pd.DataFrame(Vectorized_X)
        self.Count_Vectorizer_X_train, self.Count_Vectorizer_X_test, self.Count_Vectorizer_Y_train, self.Count_Vectorizer_Y_test = train_test_split(df_X, Count_Vectorizer_Y, test_size=0.3)
    
  
  

    def SVM_for_tfidf(self):
        # Using pipeline for applying logistic regression and one vs rest classifier
        # since the dataset is a little imbalanced, class weight is set to balanced
        self.SVM_pipeline = Pipeline([('clf', OneVsRestClassifier(SVC( kernel = 'rbf' , class_weight='balanced'))),])
        for label in self.labels:
            y_t1 = self.y_train[label]
            y_t2 = self.y_test[label]
              # Training logistic regression model on train data
            self.SVM_pipeline.fit(self.X_train, y_t1)
              # calculating test accuracy
            predictions = self.SVM_pipeline.predict(self.X_test)
            print("For ",label," Tweets: Classification report: \n")
            print(classification_report(y_t2,predictions))  


    def SVM_for_CV(self):
      self.SVM_pipeline = Pipeline([('clf', OneVsRestClassifier(SVC( kernel = 'rbf' , class_weight='balanced'))),])

      for i in self.labels:
        y_t1 = self.Count_Vectorizer_Y_train[i]
        y_t2 = self.Count_Vectorizer_Y_test[i]
        
        # Training logistic regression model on train data
        self.SVM_pipeline.fit(self.Count_Vectorizer_X_train, y_t1)
        # calculating test accuracy
        predictions = self.SVM_pipeline.predict(self.Count_Vectorizer_X_test)
        print("For ",i," Tweets: Classification report: \n")
        print(classification_report(y_t2,predictions))







    
      

      def bertEncoding(self):
        df = self.df_combined['Post']
        n_rows, n_cols = df.shape
    
        tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
        total_encode = []
        unique_encode = set()
        for post in df:
            encoded = tokenizer.encode(post)
            total_encode.append(encoded)
            for token in encoded:
                unique_encode.add(token)

        unique_encode = list(unique_encode)
        unique_encode.sort()

        arr = np.zeros((n_rows, n_cols))
        self.new_df = pd.DataFrame(arr, columns=unique_encode)
        for i in range(n_rows):
            for token in total_encode[i]:
                self.new_df.at[i, token] = 1

        return new_df





      def PCA(self):
        pca = PCA(n_components=8)
        pca.fit(self.new_df)
        self.pca_df = pca.transform(self.new_df)

      def Train_bert(self):
        y = self.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
        X_train, X_test, y_train, y_test = train_test_split(self.pca_df, y, test_size=0.30,random_state=42)

      





      # def DecisionTreeClassifier_bert(self):
        

      def train_and_evaluate_classifier(X_train, y_train, X_test, y_test, label):
          clf = DecisionTreeClassifier(class_weight='balanced', max_depth = 20, criterion='gini').fit(X_train, y_train[label])
          pred = clf.predict(X_test)
          print(f"The model accuracy for class {label}:")
          print(classification_report(pred, y_test[label]))

          labels = ['hate', 'offensive', 'non-hostile', 'defamation', 'fake']

          for label in labels:
              train_and_evaluate_classifier(X_train, y_train, X_test, y_test, label)


            
        
        
        
    
   

In [ ]:
#Making an objevt for our class
hindi_hostility_detect=Hostility()

In [ ]:
hindi_hostility_detect.df_data_test

,Unique ID,Post,Labels Set
0,1,कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...,"defamation,fake,offensive"
1,2,पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...,non-hostile
2,3,"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...",hate
3,4,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,fake
4,5,RT @_Pb_swain_: इन पंचर छापों को कोन समझाए कि ...,defamation
...,...,...,...
1648,1649,सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...,non-hostile
1649,1650,RT @ShilpiSinghINC: उमर खालिद पर तो UAPA लगा द...,hate
1650,1651,@RahulGandhi पप्पू कमीना,offensive
1651,1652,मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...,non-hostile


## Preprocessing : This involves removing mentions, URLs, hashtags, newlines and stopwords etc from the given three sets of data i.e., training, validation and testing data.

###1. Training Data

In [ ]:
hindi_hostility_detect.remove_mentions_and_URLs(hindi_hostility_detect.df_combined)
len(hindi_hostility_detect.df_combined)

5728

In [ ]:
hindi_hostility_detect.stopword_generation()

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

In [ ]:
hindi_hostility_detect.remove_hashtags(hindi_hostility_detect.df_combined)
len(hindi_hostility_detect.df_combined)

5728

In [ ]:
hindi_hostility_detect.remove_newlines(hindi_hostility_detect.df_combined)

5723    उदितराज जी हिम्मत  जुटा कर उस नेता के लिए कुछ ...
5724    उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...
5725    नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...
5726    बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...
5727    रक्षा मंत्री  ने पूर्व राष्ट्रपति  के निधन पर ...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_combined)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-441-5ab3b45419ca>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...
1    सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...
2    सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...
3     साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...
4     - अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.tokenizer_using_indic()

,Unique ID,Post,Labels Set,tokenized_post
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ..."
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा..."
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ..."
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ..."
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,..."


In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_combined)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-441-5ab3b45419ca>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...
1    सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...
2    सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...
3     साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...
4     - अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.Label_encoder_for_train()


,Unique ID,Post,Labels Set,tokenized_post,Labels,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]",0,0,1,0,1
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile],0,0,0,1,0
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile],0,0,0,1,0
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]",1,0,0,0,1
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile],0,0,0,1,0


###2. Validation Data

In [ ]:
hindi_hostility_detect.remove_mentions_and_URLs(hindi_hostility_detect.df_data_validation)
#len(hindi_hostility_detect.df_combined)

0    दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...
1    भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
2    कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
3    गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
4    यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_hashtags(hindi_hostility_detect.df_data_validation)
#len(hindi_hostility_detect.df_combined)

0    दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...
1    भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
2    कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
3    गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
4    यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_newlines(hindi_hostility_detect.df_data_validation)

806     चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से ...
807    आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने...
808    औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...
809    बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...
810    आखिरकार पॉन्डिचेरी विश्वविद्यालय के रामू नाम क...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_data_validation)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-352-df79a768acdb>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...
1    भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
2    कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
3    गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
4    यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.Label_encoder_for_validation()


,Unique ID,Post,Labels Set,Labels,defamation,fake,hate,non-hostile,offensive
0,1,दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...,non-hostile,[non-hostile],0,0,0,1,0
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[defamation],1,0,0,0,0
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile,[non-hostile],0,0,0,1,0
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile,[non-hostile],0,0,0,1,0
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile,[non-hostile],0,0,0,1,0


###3. Test Data

In [ ]:
hindi_hostility_detect.remove_mentions_and_URLs(hindi_hostility_detect.df_data_test)


0    कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1    पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...
2    कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...
3    अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4    RT  इन पंचर छापों को कोन समझाए कि उनके रोजगार ...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_hashtags(hindi_hostility_detect.df_data_test)


0    कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1    पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...
2    कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...
3    अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4    RT  इन पंचर छापों को कोन समझाए कि उनके रोजगार ...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_newlines(hindi_hostility_detect.df_data_test)

1648    सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...
1649    RT  उमर खालिद पर तो UAPA लगा दिया!कपिल मिश्रा ...
1650                                          पप्पू कमीना
1651    मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...
1652    आईपीएल में कोरोना का डर: रैना के हटने के बाद ह...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_data_test)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-352-df79a768acdb>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    कीस रोजगार नहीं कहना रोजगार नहीं मिलता 20 करोड...
1    पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...
2    कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...
3    अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4    RT  इन पंचर छापों को कोन समझाए कि उनके रोजगार ...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.Label_encoder_for_test()


,Unique ID,Post,Labels Set,Labels,defamation,fake,hate,non-hostile,offensive
0,1,कीस रोजगार नहीं कहना रोजगार नहीं मिलता 20 करोड...,"defamation,fake,offensive","[defamation, fake, offensive]",1,1,0,0,1
1,2,पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...,non-hostile,[non-hostile],0,0,0,1,0
2,3,"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ...",hate,[hate],0,0,1,0,0
3,4,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...,fake,[fake],0,1,0,0,0
4,5,RT इन पंचर छापों को कोन समझाए कि उनके रोजगार ...,defamation,[defamation],1,0,0,0,0


In [ ]:
hindi_hostility_detect

In [ ]:
hindi_hostility_detect.tf_idf_vectorizer()

In [ ]:
weight,depth,criteria="balanced",15,"gini"
hindi_hostility_detect.Decision_Tree_Classifier_for_tf_idf(weight,depth,criteria)

The model accuracy for class hate:
              precision    recall  f1-score   support

           0       0.77      0.91      0.83      1385
           1       0.50      0.26      0.34       506

    accuracy                           0.73      1891
   macro avg       0.63      0.58      0.59      1891
weighted avg       0.70      0.73      0.70      1891

The model accuracy for class offensive:
              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1474
           1       0.47      0.28      0.35       417

    accuracy                           0.77      1891
   macro avg       0.64      0.59      0.60      1891
weighted avg       0.74      0.77      0.75      1891

The model accuracy for class non-hostile:
              precision    recall  f1-score   support

           0       0.59      0.66      0.62       788
           1       0.74      0.67      0.70      1103

    accuracy                           0.67      1891
   macro av

In [ ]:
## Predictions on non hostile models
# import pickle
# model = pickle.load(open('/content/drive/MyDrive/NLP_Project/non-hostilemodel.pkl', 'rb'))

In [ ]:
hindi_hostility_detect.df_combined

,Unique ID,Post,Labels Set,tokenized_post,Labels,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]",0,0,1,0,1
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile],0,0,0,1,0
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile],0,0,0,1,0
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]",1,0,0,0,1
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
5723,5724,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,hate,"[उदितराज, जी, हिम्मत, जुटा, कर, उस, नेता, के, ...",[hate],0,0,1,0,0
5724,5725,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,non-hostile,"[उप्र, :, पीएम, रिपोर्ट, में, खुलासा, :, हार्ट...",[non-hostile],0,0,0,1,0
5725,5726,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,non-hostile,"[नौकरी, गंवा, चुके, दोस्त, की, मदद, :, नगद, के...",[non-hostile],0,0,0,1,0
5726,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake,"[बंगाल, में, हिन्दू, मरे, हैं, इसलिए, मुझे, को...",[fake],0,1,0,0,0


In [ ]:
hindi_hostility_detect.df_data_test.head(2)

,Unique ID,Post,Labels Set,Labels,defamation,fake,hate,non-hostile,offensive
0,1,कीस रोजगार नहीं कहना रोजगार नहीं मिलता 20 करोड...,"defamation,fake,offensive","[defamation, fake, offensive]",1,1,0,0,1
1,2,पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने...,non-hostile,[non-hostile],0,0,0,1,0


In [ ]:
# X = hindi_hostility_detect.df_combined['Post']
# Y = hindi_hostility_detect.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
# Count_Vectorizer_Y = hindi_hostility_detect.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]


# vectorizer = CountVectorizer(max_features=2000,min_df=2, max_df=0.8)

# # Create the vectorizer
# Vectorized_X = vectorizer.fit_transform(X).toarray()

In [ ]:
# Vectorized_X

array([[12, 38, 14, ...,  2,  4,  8],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [ ]:
preds=model.predict(Vectorized_X)

In [ ]:
# # df_data_test
# data=[]

# for post,pred in zip(hindi_hostility_detect.df_combined["Post"],preds):
#   data.append([post,pred])


In [ ]:
# save_df = pd.DataFrame(data,columns=['Posts','Hostile Prediction'])
# save_df.to_csv('/content/drive/MyDrive/NLP_Project/hostile.csv',index=False)

In [ ]:
hindi_hostility_detect.SVM_for_tfidf()

For  hate  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1630
           1       0.41      0.33      0.37       261

    accuracy                           0.84      1891
   macro avg       0.65      0.63      0.64      1891
weighted avg       0.83      0.84      0.83      1891

For  offensive  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1646
           1       0.50      0.36      0.42       245

    accuracy                           0.87      1891
   macro avg       0.70      0.65      0.67      1891
weighted avg       0.86      0.87      0.86      1891

For  non-hostile  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.78      0.77      0.78       884
           1       0.80      0.81      0.81      1007

    accuracy                          

In [ ]:
hindi_hostility_detect.Count_Vectorizer()

In [ ]:
weight,depth,criteria="balanced",15,"gini"
hindi_hostility_detect.Decision_Tree_Classifier_for_count_vectorizer(weight,depth,criteria)

The model accuracy for class hate:
              precision    recall  f1-score   support

           0       0.78      0.90      0.83      1257
           1       0.52      0.29      0.37       462

    accuracy                           0.74      1719
   macro avg       0.65      0.60      0.60      1719
weighted avg       0.71      0.74      0.71      1719

The model accuracy for class offensive:
              precision    recall  f1-score   support

           0       0.78      0.92      0.85      1279
           1       0.54      0.26      0.35       440

    accuracy                           0.75      1719
   macro avg       0.66      0.59      0.60      1719
weighted avg       0.72      0.75      0.72      1719

The model accuracy for class non-hostile:
              precision    recall  f1-score   support

           0       0.65      0.68      0.67       793
           1       0.72      0.69      0.70       926

    accuracy                           0.69      1719
   macro av

new styled func end

In [ ]:
hindi_hostility_detect.df_data_validation

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile
...,...,...,...
806,807,@ThePushpendra_ चंद फेंके हुए टुकड़ों के लिए ह...,hate
807,808,आँखों ही आँखों में इशारा हो गया \nबैठे बैठे जी...,defamation
808,809,औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...,non-hostile
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...",non-hostile


In [ ]:
#Combined dataset (test,train and validation)
hindi_hostility_detect.df_combined

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile
...,...,...,...
5723,5724,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,hate
5724,5725,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,non-hostile
5725,5726,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,non-hostile
5726,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake


In [ ]:
df_combined=hindi_hostility_detect.df_combined

In [ ]:
hindi_hostility_detect.df_data_validation

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile
...,...,...,...
806,807,@ThePushpendra_ चंद फेंके हुए टुकड़ों के लिए ह...,hate
807,808,आँखों ही आँखों में इशारा हो गया \nबैठे बैठे जी...,defamation
808,809,औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...,non-hostile
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...",non-hostile


In [ ]:
hindi_hostility_detect.stopword_generation()

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

In [ ]:
hindi_hostility_detect.remove_mentions_and_URLs(hindi_hostility_detect.df_data_validation)
len(hindi_hostility_detect.df_data_validation)

811

In [ ]:
hindi_hostility_detect.remove_mentions_and_URLs(hindi_hostility_detect.df_combined)
len(hindi_hostility_detect.df_combined)

5728

In [ ]:
hindi_hostility_detect.remove_hashtags(hindi_hostility_detect.df_data_validation)
len(hindi_hostility_detect.df_data_validation)

811

In [ ]:
hindi_hostility_detect.remove_hashtags(hindi_hostility_detect.df_combined)
len(hindi_hostility_detect.df_combined)

5728

In [ ]:
hindi_hostility_detect.remove_newlines(hindi_hostility_detect.df_data_validation)

806     चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से ...
807    आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने...
808    औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...
809    बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...
810    आखिरकार पॉन्डिचेरी विश्वविद्यालय के रामू नाम क...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_newlines(hindi_hostility_detect.df_combined)

5723    उदितराज जी हिम्मत  जुटा कर उस नेता के लिए कुछ ...
5724    उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...
5725    नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...
5726    बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...
5727    रक्षा मंत्री  ने पूर्व राष्ट्रपति  के निधन पर ...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.df_data_validation['Post']

0      दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...
1      भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
2      कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
3      गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
4      यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...
                             ...                        
806     चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से ...
807    आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने...
808    औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...
809    बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...
810    आखिरकार पॉन्डिचेरी विश्वविद्यालय के रामू नाम क...
Name: Post, Length: 811, dtype: object

In [ ]:
hindi_hostility_detect.df_combined.head(5)

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive"
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile


In [ ]:
hindi_hostility_detect.stopword_generation()

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_data_validation)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-195-dad232196991>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...
1    भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
2    कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
3    गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
4    यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.remove_stopwords(hindi_hostility_detect.df_combined)

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी',

<ipython-input-195-dad232196991>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Post"][counter]=new_lines


0    मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...
1    सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...
2    सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...
3     साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...
4     - अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...
Name: Post, dtype: object

In [ ]:
hindi_hostility_detect.tokenizer_using_indic()

,Unique ID,Post,Labels Set,tokenized_post
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ..."
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा..."
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ..."
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ..."
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,..."


In [ ]:
hindi_hostility_detect.Label_encoder_for_train()


,Unique ID,Post,Labels Set,tokenized_post,Labels,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]",0,0,1,0,1
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile],0,0,0,1,0
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile],0,0,0,1,0
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]",1,0,0,0,1
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile],0,0,0,1,0


In [ ]:
hindi_hostility_detect.df_combined

,Unique ID,Post,Labels Set,tokenized_post,Labels,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]",0,0,1,0,1
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile],0,0,0,1,0
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile],0,0,0,1,0
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]",1,0,0,0,1
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
5723,5724,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,hate,"[उदितराज, जी, हिम्मत, जुटा, कर, उस, नेता, के, ...",[hate],0,0,1,0,0
5724,5725,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,non-hostile,"[उप्र, :, पीएम, रिपोर्ट, में, खुलासा, :, हार्ट...",[non-hostile],0,0,0,1,0
5725,5726,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,non-hostile,"[नौकरी, गंवा, चुके, दोस्त, की, मदद, :, नगद, के...",[non-hostile],0,0,0,1,0
5726,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake,"[बंगाल, में, हिन्दू, मरे, हैं, इसलिए, मुझे, को...",[fake],0,1,0,0,0


In [ ]:
hindi_hostility_detect.Label_encoder_for_validation()


,Unique ID,Post,Labels Set,Labels,defamation,fake,hate,non-hostile,offensive
0,1,दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...,non-hostile,[non-hostile],0,0,0,1,0
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[defamation],1,0,0,0,0
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile,[non-hostile],0,0,0,1,0
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile,[non-hostile],0,0,0,1,0
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile,[non-hostile],0,0,0,1,0


In [ ]:
hindi_hostility_detect.df_data_validation

,Unique ID,Post,Labels Set,Labels,defamation,fake,hate,non-hostile,offensive
0,1,दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...,non-hostile,[non-hostile],0,0,0,1,0
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[defamation],1,0,0,0,0
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile,[non-hostile],0,0,0,1,0
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile,[non-hostile],0,0,0,1,0
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile,[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
806,807,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से ...,hate,[hate],0,0,1,0,0
807,808,आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने...,defamation,[defamation],1,0,0,0,0
808,809,औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...,non-hostile,[non-hostile],0,0,0,1,0
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...",non-hostile,[non-hostile],0,0,0,1,0


In [ ]:
hindi_hostility_detect.df_combined

,Unique ID,Post,Labels Set,tokenized_post,Labels,defamation,fake,hate,non-hostile,offensive
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]",0,0,1,0,1
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile],0,0,0,1,0
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile],0,0,0,1,0
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]",1,0,0,0,1
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
5723,5724,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,hate,"[उदितराज, जी, हिम्मत, जुटा, कर, उस, नेता, के, ...",[hate],0,0,1,0,0
5724,5725,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,non-hostile,"[उप्र, :, पीएम, रिपोर्ट, में, खुलासा, :, हार्ट...",[non-hostile],0,0,0,1,0
5725,5726,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,non-hostile,"[नौकरी, गंवा, चुके, दोस्त, की, मदद, :, नगद, के...",[non-hostile],0,0,0,1,0
5726,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake,"[बंगाल, में, हिन्दू, मरे, हैं, इसलिए, मुझे, को...",[fake],0,1,0,0,0


In [ ]:


hindi_hostility_detect.tf_idf_vectorizer()

In [ ]:
hindi_hostility_detect.SVM()

For  hate  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       917
           1       0.53      0.40      0.46       229

    accuracy                           0.81      1146
   macro avg       0.69      0.66      0.67      1146
weighted avg       0.79      0.81      0.80      1146

For  offensive  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       917
           1       0.53      0.40      0.46       229

    accuracy                           0.81      1146
   macro avg       0.69      0.66      0.67      1146
weighted avg       0.79      0.81      0.80      1146

For  non-hostile  Tweets: Classification report: 

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       917
           1       0.53      0.40      0.46       229

    accuracy                          

## XLM ROBERTA AND BILSTM

In [ ]:
x = hindi_hostility_detect.df_combined['Post']
def XLM_Roberta():
  tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
  total_encode=[]
  unique_encode=set()
  for i in x:
    tmp=tokenizer.encode(i)
    total_encode.append(tmp)
    for j in tmp:
      unique_encode.add(j)
  unique_encode=list(unique_encode)
  unique_encode.sort()
  arr=np.zeros((len(total_encode),len(unique_encode)))
  Encoded=pd.DataFrame(arr,columns=unique_encode)
  for k in range(0,len(total_encode)):
    for i in total_encode[k]:
      Encoded.at[k, i] = 1
  return Encoded



In [ ]:
xlm_roberta=XLM_Roberta()

Token indices sequence length is longer than the specified maximum sequence length for this model (369294 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
xlm_roberta

,0,2,3,4,5,6,7,9,10,11,...,248247,248323,248328,248344,248445,249072,249142,249143,249213,249214
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5724,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5725,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5726,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(xlm_roberta)
temp_df = pca.transform(xlm_roberta)
y_mbert = hindi_hostility_detect.df_combined[['hate', 'offensive', 'non-hostile', 'defamation', 'fake']]
X_train, X_test, y_train, y_test = train_test_split(temp_df, y_mbert, test_size=0.20,random_state=40)

from sklearn.tree import DecisionTreeClassifier
label=['hate', 'offensive', 'non-hostile', 'defamation', 'fake']
for l in label:
  clf = DecisionTreeClassifier(class_weight='balanced', max_depth = 15, criterion='gini').fit(X_train, y_train[l])
  pred=clf.predict(X_test)
  print("The model accuracy for class ",l)
  print(classification_report(pred,y_test[l]))

The model accuracy for class  hate
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       883
           1       0.49      0.25      0.34       263

    accuracy                           0.77      1146
   macro avg       0.65      0.59      0.60      1146
weighted avg       0.73      0.77      0.74      1146

The model accuracy for class  offensive
              precision    recall  f1-score   support

           0       0.86      0.91      0.89       952
           1       0.41      0.29      0.34       194

    accuracy                           0.81      1146
   macro avg       0.64      0.60      0.62      1146
weighted avg       0.79      0.81      0.80      1146

The model accuracy for class  non-hostile
              precision    recall  f1-score   support

           0       0.69      0.66      0.68       536
           1       0.71      0.74      0.73       610

    accuracy                           0.71      1146
   macro av

Implementing BiLSTM

Tensorflow Version:-- 2.9.2
Keras Version:--- 2.9.0


In [ ]:
hindi_hostility_detect.df_data_validation

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति परिपूर्ण प्रणबदा के देशहित सर...,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation
2,3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile
...,...,...,...
806,807,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से ...,hate
807,808,आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने...,defamation
808,809,औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोन...,non-hostile
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कल...",non-hostile


In [ ]:
hindi_hostility_detect.df_combined.head(2)

,Unique ID,Post,Labels Set,tokenized_post,Labels
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile]


In [ ]:
df_combined

,Unique ID,Post,Labels Set,tokenized_post,Labels
0,1,मेरे देश के हिन्दु निराले है। पक्के राम भक्त ब...,"hate,offensive","[मेरे, देश, के, हिन्दु, निराले, है, ।, पक्के, ...","[hate, offensive]"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,"[सरकार, हमेशा, से, किसानों, की, कमाई, को, बढ़ा...",[non-hostile]
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,"[सुशांत, ने, जो, बिजनेस, डील, 9, जून, को, की, ...",[non-hostile]
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive","[साले, जेएनयू, छाप, कमिने, लोग, हिन्दुओं, को, ...","[defamation, offensive]"
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी- 7 सितंबर से...,non-hostile,"[-, अनलॉक, -, 4, के, लिए, गाइडलाइन्स, जारी, -,...",[non-hostile]
...,...,...,...,...,...
5723,5724,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,hate,"[उदितराज, जी, हिम्मत, जुटा, कर, उस, नेता, के, ...",[hate]
5724,5725,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,non-hostile,"[उप्र, :, पीएम, रिपोर्ट, में, खुलासा, :, हार्ट...",[non-hostile]
5725,5726,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,non-hostile,"[नौकरी, गंवा, चुके, दोस्त, की, मदद, :, नगद, के...",[non-hostile]
5726,5727,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,fake,"[बंगाल, में, हिन्दू, मरे, हैं, इसलिए, मुझे, को...",[fake]


In [ ]:
training_label_encoded=hindi_hostility_detect.df_combined[['defamation','fake','hate','non-hostile','offensive']]
len(training_label_encoded)

5728

In [ ]:
test_label_encoded=hindi_hostility_detect.df_data_validation[['defamation','fake','hate','non-hostile','offensive']]
test_label_encoded

,defamation,fake,hate,non-hostile,offensive
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
806,0,0,1,0,0
807,1,0,0,0,0
808,0,0,0,1,0
809,0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

train_X,val_X,train_y,val_y = train_test_split(hindi_hostility_detect.df_combined['Post'],training_label_encoded,test_size = 0.25,random_state = 42)
print("Training Set Size : ",train_X.shape)
print("Validation Set Size : ",val_X.shape)

Training Set Size :  (4296,)
Validation Set Size :  (1432,)


In [ ]:
tokenizer = Tokenizer(num_words = 15000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(hindi_hostility_detect.df_combined['Post'])  
vocab_size = len(tokenizer.word_index)+1

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a Tokenizer object with the specified parameters
tokenizer = Tokenizer(num_words = 20000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(hindi_hostility_detect.df_combined['Post'])

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [ ]:

training_posts,training_labels,validation_posts,validation_labels,test_posts,test_labels = train_X,train_y,val_X,val_y,hindi_hostility_detect.df_data_validation['Post'],test_label_encoded



In [ ]:

#FINAL
path="/content/drive/MyDrive/NLP_Project/"
num_classes = 5
max_len=200
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length = max_len),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(5),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 10, return_sequences=True, dropout = 0.2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')
])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 64)           1268992   
                                                                 
 conv1d_3 (Conv1D)           (None, 196, 32)           10272     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 39, 32)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 39, 32)            0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 39, 20)           3440      
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 780)              

In [ ]:
max_len = 200
training_posts = train_X

# Training Sequences
train_sequences = tokenizer.texts_to_sequences(training_posts)
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)
training_labels = train_y

19827
Number of Training Sequences : (4296, 200)


In [ ]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences(test_posts)
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (811, 200)


In [ ]:
# Test Sequences
new_sequences = tokenizer.texts_to_sequences(hindi_hostility_detect.df_data_test['Post'])
new_pad_sequences =  keras.preprocessing.sequence.pad_sequences(new_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Blind Test Sequences :" ,new_pad_sequences.shape)

Number of Blind Test Sequences : (1653, 200)


In [ ]:
# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(validation_posts)
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

Number of Validation Sequences : (1432, 200)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert the text data to sequences of token indices
new_sequences = tokenizer.texts_to_sequences(hindi_hostility_detect.df_data_test['Post'])

# Pad the sequences to the specified maximum length
new_pad_sequences = pad_sequences(new_sequences, value=0.0, padding='post', maxlen=max_len)

# Print the shape of the padded sequences
print("Number of Blind Test Sequences :", new_pad_sequences.shape)


Number of Blind Test Sequences : (1653, 200)


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy'])
   
history = model.fit(train_pad_sequences,training_labels,validation_data=(validation_pad_sequences, validation_labels),
    epochs=10,
    batch_size = 16,
    verbose =1)

Epoch 1/10
269/269 [==============================] - 26s 67ms/step - loss: 0.1965 - accuracy: 0.7877 - val_loss: 0.4274 - val_accuracy: 0.6194
Epoch 2/10
269/269 [==============================] - 15s 55ms/step - loss: 0.1940 - accuracy: 0.7812 - val_loss: 0.4330 - val_accuracy: 0.6222
Epoch 3/10
269/269 [==============================] - 19s 69ms/step - loss: 0.1903 - accuracy: 0.7938 - val_loss: 0.4314 - val_accuracy: 0.6229
Epoch 4/10
269/269 [==============================] - 15s 55ms/step - loss: 0.1921 - accuracy: 0.7924 - val_loss: 0.4252 - val_accuracy: 0.6229
Epoch 5/10
269/269 [==============================] - 16s 58ms/step - loss: 0.1885 - accuracy: 0.7875 - val_loss: 0.4353 - val_accuracy: 0.6187
Epoch 6/10
269/269 [==============================] - 16s 60ms/step - loss: 0.1914 - accuracy: 0.7947 - val_loss: 0.4293 - val_accuracy: 0.6187
Epoch 7/10
269/269 [==============================] - 15s 56ms/step - loss: 0.1856 - accuracy: 0.7905 - val_loss: 0.4352 - val_accuracy:

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
  filepath="/content/drive/MyDrive/NLP_Project/BiLSTMModel.h5"

  model.save(filepath,save_format="h5")



Prediction on testset

In [ ]:
model = load_model(filepath)
predictions = model.predict(test_pad_sequences)
y_pred = predictions > 0.3
predicted_labels = y_pred.astype('uint8')
print("F1-Score:",f1_score(test_labels,predicted_labels,average='weighted'))
print("Accuracy:",accuracy_score(test_labels,predicted_labels))
predicted_labels

26/26 [==============================] - 1s 12ms/step
F1-Score: 0.6442957212195495
Accuracy: 0.5721331689272503


array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]], dtype=uint8)